<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_ADX_TrendScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının zaman dilimi sayısını seçmesini sağlayan fonksiyon
def get_interval_count_choice():
    print("Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?")
    print("1: Tek Zaman Dilimi")
    print("2: İki Zaman Dilimi")

    choice = input("Seçiminizi yapın (1-2): ")

    if choice == '1':
        return 1
    elif choice == '2':
        return 2
    else:
        print("Geçersiz seçim, varsayılan olarak iki zaman dilimi seçildi.")
        return 2

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice(count=2):
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimi(lerini) seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice_1 = input("Birinci zaman dilimini seçin (1-9): ")
    interval_1 = intervals.get(choice_1, ('4 Saat', Interval.in_4_hour))

    if count == 2:
        choice_2 = input("İkinci zaman dilimini seçin (1-9): ")
        interval_2 = intervals.get(choice_2, ('Günlük', Interval.in_daily))
        return [interval_1, interval_2]

    return [interval_1]

# ADX, +DI, -DI hesaplaması yapan fonksiyon
def calculate_adx(data, period=14):
    # True Range hesaplama
    data['H-L'] = data['High'] - data['Low']
    data['H-PC'] = abs(data['High'] - data['Close'].shift(1))
    data['L-PC'] = abs(data['Low'] - data['Close'].shift(1))
    data['TR'] = data[['H-L', 'H-PC', 'L-PC']].max(axis=1)

    # Yön Hareketi hesaplama
    data['+DM'] = data['High'] - data['High'].shift(1)
    data['-DM'] = data['Low'].shift(1) - data['Low']

    data['+DM'] = data['+DM'].where((data['+DM'] > data['-DM']) & (data['+DM'] > 0), 0)
    data['-DM'] = data['-DM'].where((data['-DM'] > data['+DM']) & (data['-DM'] > 0), 0)

    # ATR (Average True Range) hesaplama
    data['ATR'] = data['TR'].rolling(window=period).mean()

    # Yön Göstergeleri hesaplama
    data['+DI'] = (data['+DM'].rolling(window=period).sum() / data['ATR']) * 100
    data['-DI'] = (data['-DM'].rolling(window=period).sum() / data['ATR']) * 100

    # DX hesaplama
    data['DX'] = (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI'])) * 100

    # ADX hesaplama
    data['ADX'] = data['DX'].rolling(window=period).mean()

    return data

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Zaman dilimi sayısını seçme
interval_count = get_interval_count_choice()

# Kullanıcının seçimine göre zaman dilimlerini al
intervals = get_interval_choice(interval_count)

# Tüm sonuçları tek bir DataFrame'de toplama
df_adx_analysis = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'ADX Değeri', 'Zaman Dilimi', 'Trend Yönü', 'Kategori'])

# Seçilen zaman dilimlerine göre analiz yapar
for interval_data in intervals:
    interval_name = interval_data[0]
    interval = interval_data[1]

    for i in range(len(Hisseler)):
        try:
            data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=interval, n_bars=500)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()
            data = calculate_adx(data, period=14)
            last_signal = data.iloc[-1]
            ADX_Value = last_signal['ADX']
            Last_Price = last_signal['Close']
            Positive_DI = last_signal['+DI']
            Negative_DI = last_signal['-DI']

            # Trend Yönü
            if Positive_DI > Negative_DI:
                trend_direction = 'Pozitif'
            else:
                trend_direction = 'Negatif'

            # ADX Kategorileri
            if ADX_Value < 20:
                category = 'Zayıf trend'
            elif 20 <= ADX_Value < 25:
                category = 'Potansiyel trend'
            elif 25 <= ADX_Value < 50:
                category = 'Güçlü trend'
            elif ADX_Value >= 50:
                category = 'Çok güçlü trend'

            # Veriyi DataFrame'e ekleme
            df_adx_analysis.loc[len(df_adx_analysis)] = [Hisseler[i], Last_Price, ADX_Value, interval_name, trend_direction, category]
            print(f" {Hisseler[i]} - {interval_name} - ADX: {ADX_Value}")
        except Exception as e:
            print(f"Error processing {Hisseler[i]}: {e}")

# Öncelikle Pozitif Trend ve yüksek ADX değeri ile sıralama yapma
df_adx_analysis = df_adx_analysis.sort_values(by=['Trend Yönü', 'ADX Değeri'], ascending=[False, False])

# Excel dosyasına kaydetme
excel_file = 'adx_analysis_sorted.xlsx'
df_adx_analysis.to_excel(excel_file, index=False)

print(f"\nADX analiz sonuçları '{excel_file}' dosyasına kaydedildi.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-1son7apg
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-1son7apg
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=d2c5cbccbebf5eb486b3e6fe9c2d216c1e45960551aa3427d898594a5e7bf870
  Stored in directory: /tmp/pip-ephem-wheel-cache-05jevlsn/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00


Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?
1: Tek Zaman Dilimi
2: İki Zaman Dilimi
Seçiminizi yapın (1-2): 1
Lütfen zaman dilimi(lerini) seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Birinci zaman dilimini seçin (1-9): 7
 A1CAP - Günlük - ADX: 38.540427740105805
 ACSEL - Günlük - ADX: 29.881836371903724
 ADEL - Günlük - ADX: 14.718448118673745
 ADESE - Günlük - ADX: 22.546695813964284
 ADGYO - Günlük - ADX: 28.23618416906499
 AEFES - Günlük - ADX: 62.62281545545002
 AFYON - Günlük - ADX: 26.48317302605884
 AGESA - Günlük - ADX: 24.73449135444447
 AGHOL - Günlük - ADX: 38.62476055119041
 AGROT - Günlük - ADX: 26.483593725275178
 AGYO - Günlük - ADX: 50.31606332089239
 AHGAZ - Günlük - ADX: 19.143705916473532
 AHSGY - Günlük - ADX: 17.067772427336298
 AKBNK - Günlük - ADX: 18.536913868882827
 AKCNS - Günlük - ADX: 16.771589280315844
 AKENR - Günlük - ADX: 49.857664304963556
 AKFGY - Günlük - ADX: 11.327163103977162
